# Visualización interactiva: iPyWidgets

## Objetivos de la sesión
* Introducción a iPyWidgets
* Introducción del decorador `interact`
* Creación de sliders para las visualizaciones interactivas
* Creación de casillas desplegables para las visualizaciones interactivas
* Creación de casillas de verificación para las visualizaciones interactivas
* Creación de cuadros de texto para las visualizaciones interactivas

## Introducción
En la última lección, hablamos de cómo hacer visualizaciones interactivas basadas en Javascript usando Python y las librerías plotly y cufflinks. Esto nos proporcionó la habilidad de desplazarnos sobre los puntos de la gráfica para ver los valores, y también nos permitió incluir y excluir grupos de la visualización simplemente haciendo clic en ellos en la leyenda.

La interacción no tiene por qué limitarse a eso. En esta lección, vamos a aprender a hacer nuestras visualizaciones aún más interactivas a través del uso de widgets tales como sliders, casillas desplegables, casillas de verificación y cuadros de texto. Estos widgets van a controlar aspectos de la información que se transmiten a nuestras visualizaciones. El uso de estos widgets para cambiar valores va a hacer que las visualizaciones cambien por sí mismas. Usaremos estos widgets para ampliar la capacidad de interacción de los gráficos generados con plotly y cufflinks.

Para incorporar widgets en nuestras visualizaciones, tendremos que asegurarnos de tener instalada la biblioteca iPyWidgets.

In [ ]:
$ pip install ipywidgets

Continuemos también con la importación de todo lo que vamos a necesitar para esta lección.

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

Para esta lección continuaremos usando el mismo conjunto de datos de churn que usamos para la última lección

In [2]:
df = pd.read_csv('./data/churn.csv')

## El decorador `interact`

La biblioteca de IPyWidgets tiene varias funcionalidades para crear widgets interactivos, pero el decorador `interact` es la más sencilla y útil para empezar, por lo que en esta lección nos centraremos en ella.

El decorador `interact` acepta diferentes tipos de inputs, y el formato de esos inputs determinará el tipo de widget que se va a mostrar.

* Slider: un valor numérico, (min, max), o (min, max, step)
* Casilla desplegable: una lista o diccionario
* Casilla de verificación: Valores Verdadero o Falso
* Cuadro de texto: una cadena de caracteres entre comillas

Veremos ejemplos de cada uno de ellos en las siguientes secciones, tanto individualmente como en conjunto.

### Sliders interactivos
Uno de los tipos de widgets más útiles que podemos usar para variables numéricas es el slider. Los sliders nos permiten modificar las variables numéricas que se introducen en las visualizaciones.

Podemos ver un ejemplo básico con el siguiente histograma. Uno de los desafíos de trabajar con histogramas es determinar un número apropiado de contenedores. Los widgets pueden ayudarnos con esto permitiéndonos ver rápida y fácilmente diferentes números de contenedores sin tener que introducirlos manualmente en nuestro código y volver a generar la visualización.

Podemos utilizar el decorador interactivo para crear un widget slider en el que especificamos un rango de contenedores (desde tan sólo 8 hasta tantos como sean necesarios). Entonces, podemos simplemente definir una función hist (no importa cómo se llame) que acepte el número dinámico de ubicaciones y las conecte a un histograma para que el número se actualice a medida que se modifican los valores con el slider.

In [3]:
@interact(bins = (8, len(df['tenure'].unique())))
def hist(bins):
    df['tenure'].iplot(kind='hist', bins=bins, title='Tenure Distribution')

interactive(children=(IntSlider(value=40, description='bins', max=73, min=8), Output()), _dom_classes=('widget…

Si deslizamos el slider hacia la izquierda, obtenemos un histograma con 8 bins. Si deslizamos el slider hacia la derecha, el número de contenedores aumenta a 72, que es el número de valores de permanencia únicos que hay en nuestro conjunto de datos.

### Casillas desplegables interactivas
Otro widget muy útil cuando tenemos una visualización, y queremos ver cómo se comporta para diferentes grupos o para diferentes niveles dentro de un grupo es el cuadro desplegable. Para generarlo podemos pasarle una lista a nuestro decorador interactivo que consista en los campos que nos gustaría poder elegir. Luego podemos definir una función linechart que acepte la entrada que hemos elegido de la caja desplegable, pivotar nuestro conjunto de datos para que las columnas representen los valores categóricos en el campo elegido, y generar un gráfico de líneas con líneas que muestren los cambios en la tasa de pérdida de clientes para cada categoría.

In [4]:
df["ChurnBinary"] = df.Churn
df.ChurnBinary = df.ChurnBinary.replace('Yes',1)
df.ChurnBinary = df.ChurnBinary.replace('No',0)

In [5]:
@interact(Selection=['gender', 'SeniorCitizen', 'Partner', 
                     'Dependents', 'InternetService', 'PaymentMethod'])

def linechart(Selection):
    data = df.pivot_table(values='ChurnBinary', columns=Selection,
                            index='tenure', aggfunc='mean').reset_index()

    data.iplot(kind='line', x='tenure', xTitle='Tenure', 
               yTitle='Avg. Churn Rate', title='Avg. Churn Rate by ' + Selection.title())

interactive(children=(Dropdown(description='Selection', options=('gender', 'SeniorCitizen', 'Partner', 'Depend…

Como podemos ver, disponer de una casilla de verificación donde podamos elegir el campo que queremos visualizar nos evita tener que volver a saltar al código, crear una pivot table para el campo específico, y luego volver a generar la visualización. Esto tiene el potencial de ahorrarnos una cantidad significativa de tiempo en nuestro flujo de trabajo de exploración de datos.

Veamos otro ejemplo en el que usamos múltiples casillas de verificación para filtrar nuestros datos hasta un punto en el que podemos investigarlos a un nivel muy detallado.

A continuación vamos a crear 4 cuadros desplegables diferentes que contengan los valores categóricos únicos en los campos de gender, Partner, InternetService y PaymentMethod respectivamente. Luego definimos una función scatter donde filtramos nuestros datos por los valores elegidos en cada uno de los cuadros desplegables y luego generamos un gráfico de dispersión que muestre cuánto se cobra a cada cliente por su permanencia, añadiendo códigos de color de para definir el tipo de contrato que tiene.

In [7]:
@interact(Gender=list(df['gender'].unique()), 
          Partner=list(df['Partner'].unique()),
          Internet=list(df['InternetService'].unique()), 
          Payment=list(df['PaymentMethod'].unique())
         )

def scatter(Gender, Partner, Internet, Payment):
    data = df[(df['gender']==Gender) & 
              (df['Partner']==Partner) & 
              (df['InternetService']==Internet) & 
              (df['PaymentMethod']==Payment)]

    data.iplot(kind='scatter', x='tenure', y='MonthlyCharges', 
               categories='Contract', text='customerID', 
               xTitle='Tenure', yTitle='Monthly Charges',
               title='Charges vs. Tenure')

interactive(children=(Dropdown(description='Gender', options=('Female', 'Male'), value='Female'), Dropdown(des…

Debemos tener en cuenta que hemos añadido un argumento `text` a nuestro método `iplot` que nos mostrará el ID de cada cliente además de la cantidad mensual que está pagando cuando pasamos el puntero del ratón por encima de los nodos

### Casillas de verificación interactivas
Las casillas de verificación interactivas también pueden ayudarnos a explorar un conjunto de datos, especialmente cuando hay campos binarios cuyo impacto nos gustaría visualizar. La manera de hacer esto es mapear si la casilla de verificación está marcada con una condición correspondiente. Una vez hecho esto, sólo tenemos que filtrar el conjunto de datos en función de esas condiciones.

A continuación hay un ejemplo que hace exactamente esto. El decorador interact tiene dos argumentos Verdadero/Falso que se mostrarán como casillas de verificación - una para Senior y otra para PhoneService. Dentro de nuestra función de gráfico de barras, escribimos algunas declaraciones condicionales que traducirán esas opciones Verdadero/Falso en condiciones que podemos usar para filtrar nuestros datos. A continuación, aplicamos estos filtros y agrupamos los datos por Método de Pago, calculando la tasa de abandono media para cada uno de ellos. Finalmente, generamos un gráfico de barras que muestra la tasa de abandono promedio para cada método de pago basado en los filtros aplicados a través de las casillas de verificación.

In [11]:
@interact(Senior=False, PhoneService=True)

def barchart(Senior, PhoneService):
    if Senior==True:
        senior = df['SeniorCitizen']==1
    else:
        senior = df['SeniorCitizen']==0
    
    if PhoneService==True:
        phone = df['PhoneService']=='Yes'
    else:
        phone = df['PhoneService']=='No'
    
    data = df[(senior) & (phone)].groupby('PaymentMethod').agg({'ChurnBinary':'mean'}).reset_index()
    
    data.iplot(kind='bar', x='PaymentMethod', xTitle='Payment Method',
               yTitle='Avg. Churn Rate', color='blue', 
               title='Churn Rate by Payment Method')

interactive(children=(Checkbox(value=False, description='Senior'), Checkbox(value=True, description='PhoneServ…

De forma predeterminada, la casilla PhoneService está marcada porque inicialmente la definimos como True.

### Cuadros de texto interactivos
El último tipo de widget que vamos a tratar en esta lección es el cuadro de texto interactivo. Estos son buenos para filtrar cuando tenemos variables categóricas que son diferentes pero que tienen algun string en común. Por ejemplo, el campo PaymentMethod en nuestro conjunto de datos tiene cuatro valores únicos.

In [13]:
df['PaymentMethod'].unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

Hay similitudes entre los pares de estas categorías únicas - dos de ellas tienen la cadena 'check' en común y las otras dos tienen la cadena 'automatic' en común. Podemos usar un cuadro de texto para proporcionar un medio flexible mediante el cual podemos visualizar cualquiera de estas opciones en base a las cadenas únicas que contienen, o grupos de ellas en base a sus cadenas co-existentes.

Para ello, volveremos a utilizar el decorador `interact`, pasando un argumento `Payment` con una cadena en blanco. Esto creará un cuadro de texto en el que podremos escribir cualquier cadena que queramos. En nuestra función de gráficos, filtraremos nuestro conjunto de datos sólo a los métodos de pago que contienen la cadena escrita en el cuadro de texto. Luego agregamos, clasificamos y visualizamos como un gráfico de barras.

In [17]:
New = range(0,10)
Regular = range(10,30)
Loyal = range(30,56)
VeryLoyal = range(56,73)
df["TenureLevel"] = df.tenure
df.TenureLevel = df.TenureLevel.replace(New,'New')
df.TenureLevel = df.TenureLevel.replace(Regular,'Regular')
df.TenureLevel = df.TenureLevel.replace(Loyal,'Loyal')
df.TenureLevel = df.TenureLevel.replace(VeryLoyal,'VeryLoyal')

In [18]:
@interact(Payment='')

def chart(Payment):
    data = df[df['PaymentMethod'].str.contains(Payment)]
    data = data.groupby('TenureLevel').agg({'MonthlyCharges':'sum'}).reset_index()
    custom_dict = {'New': 0, 'Regular': 1, 'Loyal': 2, 'Very Loyal' : 3}  
    data = data.iloc[data['TenureLevel'].map(custom_dict).argsort()].set_index('TenureLevel')
    
    data.iplot(kind='bar', xTitle='Values')

interactive(children=(Text(value='', description='Payment'), Output()), _dom_classes=('widget-interact',))

Si dejamos el cuadro de texto vacío, el gráfico nos muestra que el total de los cargos mensuales que cobra la compañía no depende demasiado del nivel de antiguedad si no tenemos en cuenta el método de pago

Cuando escribimos'check' en el cuadro de texto, el gráfico se actualiza y nos muestra el total de los Cargos Mensuales que la compañía cobra de cada nivel de antigüedad solo para los métodos de pago concretos que contienen la cadena "check". Parece que los clientes menos leales tienden a utilizar más este método de pago.

Sin embargo, cuando escribimos 'auto' en el cuadro de texto, el gráfico muestra que los clientes más fieles tienden a utilizar estos métodos de pago para pagar sus facturas.

## Resumen
En esta lección, hemos introducido iPyWidgets y cómo podemos utilizarlos para añadir fácilmente más interactividad a nuestras visualizaciones. Aprendimos sobre el poderoso decorador `interact` y tratamos ejemplos de cada tipo de widget (sliders, casillas desplegables, casillas de verificación y cuadros de texto). Hemos cubierto cada uno de ellos individualmente, pero no hay nada que nos impida combinar varios widgets para crear visualizaciones muy complejas e interactivas. De hecho, te animamos a que te desafíes a ti mismo a hacerlo. Ya disponemos de todas las herramientas que necesitamos en nuestro arsenal, y la incorporación de widgets en nuestro flujo de trabajo analítico nos permite explorar los datos con mayor facilidad y, al mismo tiempo, ahorrar una cantidad significativa de tiempo.